In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
datadir = '/home/mei/nas/docker/dataset/EICU/eicu-collaborative-research-database-2.0/'
porcesseddir = '/home/mei/nas/docker/processedData/'

In [3]:
def round_up(x, base=5):
    return base * round(x/base)

In [48]:
df = pd.read_csv(datadir + 'vitalAperiodic.csv.gz', compression='gzip')

In [49]:
df.head()

,vitalaperiodicid,patientunitstayid,observationoffset,noninvasivesystolic,noninvasivediastolic,noninvasivemean,paop,cardiacoutput,cardiacinput,svr,svri,pvr,pvri
0,4295739,141168,349,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4295737,141168,123,106.0,68.0,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4295741,141168,1398,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4295740,141168,441,NaN,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4295738,141168,138,111.0,62.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df_patient = pd.read_csv(porcesseddir + 'selected_patients.csv')
df_selected_vitalA= pd.merge(df_patient, df, on='patientunitstayid')
df_selected_vitalA.head()

,patientunitstayid,hospitaladmitoffset,gender,age,apacheadmissiondx,unitadmitsource,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus,...,noninvasivesystolic,noninvasivediastolic,noninvasivemean,paop,cardiacoutput,cardiacinput,svr,svri,pvr,pvri
0,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired,...,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired,...,106.0,68.0,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired,...,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired,...,NaN,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141168,0,Female,70,"Rhythm disturbance (atrial, supraventricular)",Direct Admit,84.3,85.8,Death,Expired,...,111.0,62.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df=df_selected_vitalA.copy()
# Set indices
df.set_index(['patientunitstayid', 'observationoffset'], inplace = True)
df.drop(df.index[np.where(df.index.get_level_values('observationoffset') < 0)], inplace= True)
df.drop(columns = 'vitalaperiodicid', inplace= True)
df.sort_index(level = ['patientunitstayid', 'observationoffset'], inplace = True)
columns_to_drop = ['hospitaladmitoffset','gender','age','apacheadmissiondx','unitadmitsource','admissionweight','dischargeweight','unitdischargelocation','unitdischargestatus'] 
df= df.drop(columns=columns_to_drop)
# Resample every 5 mins
df.rename(round_up, level = 'observationoffset', inplace = True)
# df = df.groupby(['patientunitstayid', 'observationoffset']).mean()

df.head()

noninvasivesystolic  \
patientunitstayid observationoffset                        
141168            125                              106.0   
                  140                              111.0   
                  350                                NaN   
                  440                                NaN   
                  1400                               NaN   

                                     noninvasivediastolic  noninvasivemean  \
patientunitstayid observationoffset                                          
141168            125                                68.0             81.0   
                  140                                62.0             82.0   
                  350                                 NaN             79.0   
                  440                                 NaN             62.0   
                  1400                                NaN             27.0   

                                     paop  cardiacoutput  cardiacinput  svr  \
patientunitstayid observationoffset                                           
141168            125                 NaN            NaN           NaN  NaN   
                  140                 NaN            NaN           NaN  NaN   
                  350                 NaN            NaN           NaN  NaN   
                  440                 NaN            NaN           NaN  NaN   
                  1400                NaN            NaN           NaN  NaN   

                                     svri  pvr  pvri  
patientunitstayid observationoffset                   
141168            125                 NaN  NaN   NaN  
                  140                 NaN  NaN   NaN  
                  350                 NaN  NaN   NaN  
                  440                 NaN  NaN   NaN  
                  1400                NaN  NaN   NaN

In [18]:
# df.to_csv(porcesseddir + 'selected_vitalAperiodic.csv',  index=True)
# print("Data exported successfully to 'selected_vitalAperiodic.csv'.")

Data exported successfully to 'selected_vitalAperiodic.csv'.


In [52]:
selected_vAp_p=len(list(df.index.get_level_values('patientunitstayid').unique()))
selected_vAp_x=len(df)
print("There are {} unique patientunitstayid in the selected patients with the total {} records.".format(selected_vAp_p,selected_vAp_x))

There are 98981 unique patientunitstayid in the selected patients with the total 12212644 records.


In [53]:
missing_ratio = pd.DataFrame(df.isna().sum() / len(df.index), columns = ['missing ratio'])
missing_ratio

,missing ratio
noninvasivesystolic,0.053537
noninvasivediastolic,0.053419
noninvasivemean,0.047889
paop,0.999434
cardiacoutput,0.972095
cardiacinput,0.985574
svr,0.979530
svri,0.996826
pvr,0.999808
pvri,0.999808


In [54]:
def is_dead(x):
    if x == 'EXPIRED':
        return True
    return False

In [55]:
mean = df.groupby(['patientunitstayid']).mean()
print('Mean value of vital signs over ICU stay for each patient')
mean.head()

Mean value of vital signs over ICU stay for each patient


,noninvasivesystolic,noninvasivediastolic,noninvasivemean,paop,cardiacoutput,cardiacinput,svr,svri,pvr,pvri
patientunitstayid,,,,,,,,,,
141168,108.500000,65.000000,66.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141208,124.523810,63.190476,86.285714,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141229,124.454545,61.818182,85.727273,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141260,113.687500,74.000000,87.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141264,137.000000,83.000000,103.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
std = df.groupby(['patientunitstayid']).std()
print('Standard deviation of vital signs over ICU stay for each patient')
std.head()

Standard deviation of vital signs over ICU stay for each patient


,noninvasivesystolic,noninvasivediastolic,noninvasivemean,paop,cardiacoutput,cardiacinput,svr,svri,pvr,pvri
patientunitstayid,,,,,,,,,,
141168,3.535534,4.242641,23.381617,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141208,11.530911,7.934854,9.386921,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141229,12.832487,4.935953,6.739301,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141260,11.187605,8.485281,9.831921,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df.describe()

,noninvasivesystolic,noninvasivediastolic,noninvasivemean,paop,cardiacoutput,cardiacinput,svr,svri,pvr,pvri
count,1.155881e+07,1.156025e+07,1.162779e+07,6907.000000,340797.000000,176184.000000,249990.000000,38758.000000,2347.000000,2340.000000
mean,1.218155e+02,6.600466e+01,8.184765e+01,23.211235,2.703065,3.205689,972.008316,1772.505496,152.550490,295.334188
std,2.509533e+01,1.651966e+01,1.834896e+01,10.242490,2.918017,31.797766,416.044680,807.991492,152.120805,314.506976
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,-7105.000000,-13390.000000,-953.000000,-1754.000000
25%,1.040000e+02,5.500000e+01,6.900000e+01,17.000000,0.000000,0.000000,699.000000,1362.000000,84.000000,162.000000
50%,1.190000e+02,6.400000e+01,8.000000e+01,17.000000,2.200000,0.000000,910.000000,1694.000000,130.000000,245.000000
75%,1.380000e+02,7.500000e+01,9.300000e+01,30.000000,5.200000,0.000000,1177.000000,2135.000000,189.000000,378.000000
max,2.920000e+02,2.770000e+02,2.860000e+02,97.000000,19.799999,327.630000,7810.000000,12915.000000,3707.000000,7237.000000
